In [ ]:
using DifferentialEquations
using PyPlot

In [ ]:
cd("../rms")
include("Parse.jl")
include("Reactor.jl")

In [ ]:
phaseDict = readinput("testing/superminimal.yml") #load mechanism dictionary
spcs = phaseDict["gas"]["Species"] #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["gas"]["Reactions"]

In [ ]:
ig = IdealGas(spcs,rxns,name="gas") #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>1000.0,"P"=>1e5,"H2"=>0.67,"O2"=>0.33]) #Set simulation Initial Temp and Pressure
state = MolarState(initialconds,ig) #Define the initial state of the system
domain = ConstantTPDomain(state=state,phase=ig) #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = BatchSingleDomainReactor(domain,(0.0,141.0)) #Create the reactor object
sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-12) #solve the ode associated with the reactor

In [ ]:
ts = exp.(range(log(1e-15),length=1000,stop=log(140.0))) #logspace for Julia
plot(ts,[sol(t)./sum(sol(t)) for t in ts]) #plot mole fractions of all species
legend([sp.name for sp in ig.species])